In [1]:
# | default_exp classes.DomoApplication_Job

A job refers to a scheduled task in an application.

For the purposes of this beta class, the only Job type implemented has been Watchdog.  This class will be heavily revised in a future development sprint.

An additional Job type might be PDP Automation Toolkit.

In [2]:
# | exporti
from dataclasses import dataclass, field
from typing import List, Optional

import httpx

from nbdev.showdoc import patch_to
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import datetime  as dt

from pprint import pprint

import domolibrary.routes.application as application_routes

import domolibrary.utils.convert as cc

In [3]:
# |hide
import os

In [4]:
# | export

@dataclass
class DomoTrigger_Schedule:
    schedule_text: str = None
    schedule_type: str = "scheduleTriggered"

    minute: int = None
    hour: int = None
    minute_str: str = None
    hour_str: str = None

    @classmethod
    def _from_str(cls, s_text, s_type):
        sched = cls(schedule_type=s_type, schedule_text=s_text)

        try:
            parsed_hour = s_text.split(" ")[2]
            parsed_minute = s_text.split(" ")[1]

            if "*" in parsed_hour or "/" in parsed_hour:
                sched.hour_str = parsed_hour
            else:
                sched.hour = int(float(parsed_hour))
            if "*" in parsed_minute:
                sched.minute_str = parsed_minute
            else:
                sched.minute = int(float(parsed_minute))

            return sched

        except Exception as e:
            print(f"unable to parse schedule {s_text}")
            print(e)

    def to_obj(self):
        return {"hour": int(self.hour), "minute": int(self.minute)}

    def to_schedule_obj(self):
        minute = self.minute_str if self.minute_str is not None else str(self.minute)
        hour = self.hour_str if self.hour_str is not None else str(self.hour)
        return {
            "eventEntity": f"0 {minute} {hour} ? * *",
            # old value on Jan 13
            # "eventEntity": f'0 {minute} {hour} 1/1 * ? *',
            "eventType": self.schedule_type,
        }


@dataclass
class DomoTrigger:
    id: str
    job_id: str
    schedule: List[DomoTrigger_Schedule] = None



@dataclass
class DomoJob:
    auth : dmda.DomoAuth = field(repr = False)
    id: str
    name: str
    description: str
    user_id: str
    application_id: str
    customer_id: str
    execution_timeout: int
    
    execution_payload: dict
    share_state: dict
    created_dt: dt.datetime
    updated_dt :dt.datetime
    
    triggers : List[DomoTrigger] = field(default_factory=[])

    
    # entity_ids: list
    # remote_instance: str
    # job_type: str
    # entity_type: str
    # max_indexing_time_min: int
    # variance_percent: int
    # min_update_frequency_min: int
    # sql_query: str
    # notify_user_ids: list
    # metrics_dataset_id: str
    # notify_group_ids: list
    # notify_email_addressess: list
    # resources_requests: str
    # resources_limits: str

    triggers: List[DomoTrigger] = None

    def _generate_base_json(obj):
            return {
            "id":obj['jobId'],
            "application_id" : obj['applicationId'],
            "customer_id" : obj['customerId'],
            "name"  : obj['jobName'],
            "description" : obj['jobDescription'],
            "user_id" : obj['userId'],
            "execution_timeout" : obj['executionTimeout'],
            "execution_payload": obj['executionPayload'],
            "share_state" : obj['shareState'],
            'created_dt' : cc.convert_epoch_millisecond_to_datetime(obj['created']),
            'updated_dt' : cc.convert_epoch_millisecond_to_datetime(obj['updated'])
            
            
            # entity_ids=(
            #     dd.executionPayload.watcherParameters.entityIds
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # job_type=(
            #     dd.executionPayload.watcherParameters.type
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # entity_type=(
            #     dd.executionPayload.watcherParameters.entityType
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # max_indexing_time_min=(
            #     dd.executionPayload.watcherParameters.maxIndexingTimeInMinutes
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # variance_percent=(
            #     dd.executionPayload.watcherParameters.variancePercent
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # min_update_frequency_min=(
            #     dd.executionPayload.watcherParameters.minDataUpdateFrequencyInMinutes
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # sql_query=(
            #     dd.executionPayload.watcherParameters.sqlQuery
            #     if dd.executionPayload.watcherParameters
            #     else []
            # ),
            # "notify_user_ids" : obj['executionPayload']['notifyUserIds'],
            # "notify_group_ids" : obj['executionPayload']['notifyGroupIds'],
            # "notify_email_addressess":obj['executionPayload']['notifyEmailAddresses'],
            # "metrics_dataset_id":obj['executionPayload']['metricsDatasetId'],
            # "resources_requests": obj['resources']['requests']['memory'],
            # "resources_limits":  obj['resources']['limits']['memory']
    }


    @classmethod
    def _from_json(cls, obj, auth):
        triggers_ls = obj.get("triggers", None)

        triggers_dj = [
            DomoTrigger(
                id=tg.get("triggerId"),
                job_id=tg.get("jobId"),
                schedule=DomoTrigger_Schedule._from_str(
                    s_text=tg.get("eventEntity"), s_type=tg.get("eventType")
                ),
            )
            for tg in triggers_ls
        ] if triggers_ls else []

        base_obj = cls._generate_base_json(obj)

        return cls(
            auth = auth,
            ** base_obj,
            triggers = triggers_dj,
        )

    def to_json(self):
        body = {
            "jobName": self.name,
            "jobDescription": self.description,
            "executionTimeout": self.execution_timeout,
            "accounts": [],
            "executionPayload": {
                "notifyUserIds": self.notify_user_ids or [],
                "notifyGroupIds": self.notify_group_ids or [],
                "notifyEmailAddresses": self.notify_email_addressess or [],
                "watcherParameters": {
                    "entityIds": self.entity_ids,
                    "type": self.job_type,
                    "entityType": self.entity_type,
                    "maxIndexingTimeInMinutes": self.max_indexing_time_min,
                    "variancePercent": self.variance_percent,
                    "sqlQuery": self.sql_query,
                },
                "metricsDatasetId": self.metrics_dataset_id,
            },
            "resources": {
                "requests": {"memory": self.resources_requests},
                "limits": {"memory": self.resources_limits},
            },
            "triggers": (
                [self.triggers[0].schedule.to_schedule_obj()]
                if len(self.triggers) > 0
                else []
            ),
        }
        return body

In [5]:
@patch_to(DomoJob, cls_method=True)
async def _get_by_id(
    cls,
    application_id,
    job_id,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    debug_num_stacks_to_drop=2,
):

    return  await application_routes.get_application_job_by_id(
        auth=auth,
        application_id=application_id,
        job_id = job_id,
        session=session,
        debug_api=debug_api,
    
        parent_class=cls.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

@patch_to(DomoJob, cls_method= True)
async def get_by_id(
        cls,
        application_id,
        job_id,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        session: Optional[httpx.AsyncClient] = None,
        debug_num_stacks_to_drop=2,
        return_raw :bool = False
    ):
        res = await cls._get_by_id(
            application_id = application_id,
            job_id = job_id,
            auth = auth,
            debug_api = debug_api,
            session = session,
            debug_num_stacks_to_drop= debug_num_stacks_to_drop)

        if return_raw:
            return res
        
        return cls._from_json(
            obj = res.response,
            auth = auth
        )
    

In [6]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoJob.get_by_id(
    job_id= '743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
    application_id = 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
    auth = token_auth,
)

DomoJob(id='743c1c6f-80d5-4b47-b02e-0ea28f6a5683', name='tag_inactive_owners', description='t', user_id=1893952720, application_id='a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4', customer_id='mmmm-0012-0200', execution_timeout=1440, execution_payload={'emails': ['jae@onyxreporting.com'], 'resourceTypes': ['datasource'], 'tagFilter': [], 'ownerFilter': [], 'typeFilter': [], 'domain': None, 'taggingPolicies': ['ownerInactive'], 'metricsDatasetId': '29a3417d-1543-46a3-abd1-cbe84f257fb5'}, share_state={'sharedEntities': [{'id': '1893952720', 'type': 'USER', 'accessLevel': 'OWNER'}]}, created_dt=datetime.datetime(2024, 2, 21, 11, 21, 31), updated_dt=datetime.datetime(2024, 2, 26, 13, 42, 17), triggers=[])

In [7]:
#| export

@dataclass
class DomoJob_RemoteDomoStats(DomoJob):
    remote_instance : str = None

    @classmethod 
    def _from_json( cls, obj, auth):
        base_class = cls._generate_base_json(obj)

        return cls(
            base_class, 
            remote_instance = cls._get_remote_instance(obj.remoteInstance)
        )
                
    @staticmethod
    def _get_remote_instance(remote_instance):
        return remote_instance.replace(".domo.com", "")
    
    @classmethod
    async def get_by_id(
        cls,
        application_id,
        job_id,
        auth: dmda.DomoAuth,
        debug_api: bool = False,
        session: Optional[httpx.AsyncClient] = None,
        debug_num_stacks_to_drop=2,
        return_raw :bool = False
    ):
        res = await cls._get_by_id(
            application_id = application_id,
            job_id = job_id,
            auth = auth,
            debug_api = debug_api,
            session = session,
            debug_num_stacks_to_drop= debug_num_stacks_to_drop)

        if return_raw:
            return res
        
        cls._from_json(
            obj = res.response,
            auth = auth
        )
    

In [8]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

token_auth = dmda.DomoTokenAuth(
    domo_instance= os.environ['BETA_INSTANCE'],
    domo_access_token=os.environ["BETA_ACCESS_TOKEN"],
)

await DomoJob_RemoteDomoStats.get_by_id(
    job_id= '743c1c6f-80d5-4b47-b02e-0ea28f6a5683',
    application_id = 'a99c3fd8-a0f6-4d06-9a1d-74f3d12293d4',
    auth = token_auth,
    return_raw = False
)

RouteFunction_ResponseTypeError: Expected function to return an instance of ResponseGetData got <class 'domolibrary.routes.application.ApplicationError_NoneRetrieved'> instead.  Refactor function to return ResponseGetData class

In [ ]:

@patch_to(DomoJob)
async def update(
    self : DomoJob ,
    body: dict,
    job_id: str,
    application_id: str,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
    debug_num_stacks_to_drop = 2
):
    
    res = await application_routes.update_job(
        auth=self.auth,
        body=body,
        application_id=application_id,
        job_id=job_id,
        debug_api=debug_api,
        session=session,
        parent_class = self.__class__.__name__,
        debug_num_stacks_to_drop = debug_num_stacks_to_drop
    )

    return res

In [ ]:
# |exporti

@patch_to(DomoJob)
async def execute(
    self,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
):

    res = await application_routes.execute_job(
        auth=self.auth,
        application_id=self.application_id,
        job_id=self.job_id,
        debug_api=debug_api,
        session=session,
    )

    return res

In [ ]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await application_routes.get_applications(auth=token_auth)
application_id = next(
    (
        app["applicationId"]
        for app in res.response
        if app["name"] == "Toolkit: DataSet Tag Automation"
    )
)

res = await application_routes.get_application_jobs(application_id=application_id, auth=token_auth)
job_id = next((job["jobId"] for job in res.response))

job_id

await execute_application_job(
    auth=token_auth, application_id=application_id, job_id=job_id
)

NameError: name 'execute_application_job' is not defined

In [ ]:
@classmethod
async def create_domostats_job(
    cls,
    auth: dmda.DomoFullAuth,
    domostats_schedule: DomoTrigger_Schedule,
    application_id: str,
    target_instance: str,
    report_dict: dict,
    output_dataset_id: str,
    account_id: str,
    execution_timeout: int = 1440,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_num_stacks_to_drop=2,
    session: Optional[httpx.AsyncClient] = None,
):

    schedule_obj = domostats_schedule.to_schedule_obj()

    body = application_routes.generate_body_remote_domostats(
        target_instance=target_instance,
        report_dict=report_dict,
        output_dataset_id=output_dataset_id,
        account_id=account_id,
        schedule_ls=[schedule_obj],
        execution_timeout=execution_timeout,
    )

    res = await application_routes.add_job(
        auth=auth,
        application_id=application_id,
        body=body,
        debug_api=debug_api,
        session=session,
        parent_class=self.__class__.__name__,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if return_raw:
        return res

    if res.status != 200:
        return False

    return True

@classmethod
async def create_watchdog_job(
    cls,
    auth: dmda.DomoFullAuth,
    body: str,
    application_id: str,
    debug_api: bool = False,
    session: Optional[httpx.AsyncClient] = None,
):

    res = await application_routes.add_job(
        auth=auth,
        application_id=application_id,
        body=body,
        debug_api=debug_api,
        session=session,
    )
    if debug_api:
        print(res)

    if res.status != 200:
        return False

    return True

@classmethod
async def generate_watchdog_body(
    cls,
    watchdog_report_type: WatchDogType,
    watchdog_schedule: DomoTrigger_Schedule,
    job_name: str,
    notify_user_ids_ls: list,
    notify_group_ids_ls: list,
    notify_emails_ls: list,
    entity_ids_ls: list,
    entity_type: str,
    metric_dataset_id: str,
    sql_query: str = None,
    variance_percent: int = 30,
    max_indexing_time_mins: int = 30,
    execution_timeout: int = 1440,
    min_update_frequency_min: int = 1440,
):
    schedule_obj = watchdog_schedule.to_schedule_obj()

    body = application_routes.generate_body_watchdog_generic(
        job_name=job_name,
        notify_user_ids_ls=notify_user_ids_ls,
        notify_group_ids_ls=notify_group_ids_ls,
        notify_emails_ls=notify_emails_ls,
        entity_ids_ls=entity_ids_ls,
        entity_type=entity_type,
        metric_dataset_id=metric_dataset_id,
        schedule_ls=[schedule_obj],
        job_type=watchdog_report_type.value,
        execution_timeout=execution_timeout,
    )

    if watchdog_report_type == watchdog_report_type.DATASET_INDEX_TIME:
        child = {"maxIndexingTimeInMinutes": max_indexing_time_mins}
        body["executionPayload"]["watcherParameters"].update(child)

    if (
        watchdog_report_type == watchdog_report_type.ROW_COUNT_CHANGE
        or watchdog_report_type == watchdog_report_type.OUTLIER_EXECUTION
    ):
        child = {"variancePercent": variance_percent}
        body["executionPayload"]["watcherParameters"].update(child)

    if watchdog_report_type == watchdog_report_type.CUSTOM_QUERY:
        child = {"sqlQuery": sql_query}
        body["executionPayload"]["watcherParameters"].update(child)

    if watchdog_report_type == watchdog_report_type.LAST_UPDATED_DATA:
        child = {"minDataUpdateFrequencyInMinutes": min_update_frequency_min}
        body["executionPayload"]["watcherParameters"].update(child)

    return body



NameError: name 'WatchDogType' is not defined